In [1]:
import os
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [2]:
path_P658207LE0A_SEGMENT_Model = "./../models/P658207LE0A_SEGMENT.pt"
P658207LE0A_SEGMENT_Model = YOLO(path_P658207LE0A_SEGMENT_Model)

In [3]:
image_path = "./../inspection/658207LE0A/20240917_142316.png"
image = cv2.imread(image_path)

In [4]:
result = P658207LE0A_SEGMENT_Model(image_path, imgsz=960, save=True, visualize=False, show_boxes=False)


image 1/1 c:\Users\sekkei\Documents\aikensadcam\aikensa_dcam\aikensa\tools\..\inspection\658207LE0A\20240917_142316.png: 192x960 1 WS, 115.2ms
Speed: 3.2ms preprocess, 115.2ms inference, 145.0ms postprocess per image at shape (1, 3, 192, 960)
Results saved to runs\segment\predict3
